In [ ]:
import numpy as np

import os

import matplotlib.pyplot as plt

from tensorflow.keras.models import model_from_json
from scipy.stats import pearsonr

from mask_utils import show_image_with_masks,iou,symmetric_hausdorff_distance,mean_contour_distance,dsc,normalise_image

from network_utils import gpu_memory_limit

from MultiResUNet.MultiResUNet import MultiResUnet

import tensorflow as tf

import itertools

from sklearn.linear_model import LinearRegression

import pickle

import pandas as pd

In [ ]:
#limit how much GPU RAM can be allocated by this notebook... 8GB is 1/3 of available
# gpu_memory_limit(5000)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
#graph outputs...
DataDir = './data/pericardial/wsx_round2/'

splitDataFile = os.path.join(DataDir,'splitData.pickle')

if os.path.isfile(splitDataFile):
    splitData = pickle.load(open(splitDataFile,'rb'))
    X, X_test, Y, Y_test,pxArea,pxArea_test,pxSpacing,pxSpacing_test = splitData
    
else:
    from sklearn.model_selection import train_test_split

    #load data - these files created by extract_dcm_for_wsx.ipynb
    X = np.load(os.path.join(DataDir,'X.npy'))
    Y = np.load(os.path.join(DataDir,'Y.npy')).astype('float')
    pxArea = np.load(os.path.join(DataDir,'pxSize.npy'))
    pxSpacing = np.sqrt(pxArea)

    #ensure the shape is correct arrays saved were rank 3, so this changes to rank 4 (last dimension represents channels)
    X = X.reshape([*X.shape,1])
    Y = Y.reshape([*Y.shape,1])

    #do train/test split!
    splitData = train_test_split(X, Y, pxArea,pxSpacing, test_size=0.2,random_state=101)
    pickle.dump(splitData,open(splitDataFile,'wb'))
    #extract individual bits
    X, X_test, Y, Y_test,pxArea,pxArea_test,pxSpacing,pxSpacing_test = splitData

del splitData #as this variable is o longer required   

M = X.shape[0]
MTest = X_test.shape[0]
imShape = (1,*X.shape[1:])

PXSPACING = pickle.load(open(os.path.join('data','PXSPACING.pickle'),'rb'))
PXAREA = np.product(PXSPACING)

RESULTFILE = os.path.join('data','models','model_history.csv')


In [ ]:
#load the model
modelBaseName = 'mrunet_bayesian_2020-07-13_13:40' 

#directory for graph outputs.
graphDir = os.path.join('graphs','performance',modelBaseName)
if not os.path.isdir(graphDir):
    os.makedirs(graphDir)

#location of the actual saved model
modelBaseName = os.path.join('data','models',modelBaseName)

modelParamFile = modelBaseName + '.h5'
modelArchitecture = modelBaseName + '.json'

with open( modelArchitecture , 'r') as json_file:
    model = model_from_json( json_file.read() )

model.load_weights(modelParamFile)

In [ ]:
#FUNCTIONS FOR DOING STOCHASTIC PREDICTIONS...

#FIXMMEEEEEEEE make it so these can be called on arrays where M>1!!!!! BECAUSE THIS SUCKS

def global_iou(predictions):
    
    '''takes the iou of multiple different segmentations'''
    
    intersection = np.min(predictions,axis=0).sum()
    union = np.max(predictions,axis=0).sum()
    
    return intersection / union

def global_dsc(predictions):
    
    N = predictions.shape[0]
    numerator = N * np.min(predictions,axis=0).sum()
    denominator = predictions.sum()
    
    return numerator/denominator
    
def mean_pairwise_iou(predictions):
    
    #all combinations of inputs
    ious = [iou(a,b) for a,b in itertools.combinations(predictions,2)]
    
    return np.mean(ious)

def mean_pairwise_dsc(predictions):
    
    #all combinations of samples, which will be axis 0
    dscs = np.array([dsc(a,b) for a,b in itertools.combinations(predictions,2)])
    
    dscs[np.isnan(dscs)] = 0
    
    return np.mean(dscs)
    
def voxel_uncertainty(predictions):
    
    '''voxel-wise uncertainty as defined in Roy et al (2018)'''
    
    #strcture-and-voxel-wise uncertainty (compresses over the sample axis
    feature_uncertainty = -np.sum(predictions*np.log(predictions),axis = 0)
    #global uncertainty is the sum over the feature axis
    global_uncertainty = np.sum(feature_uncertainty,axis=-1)
    
    return global_uncertainty
    
def mean_std_area(predictions):
    
    '''the area occupied by each segmented channel. outputs two array: mean and standard deviation
    RETURNS ANSWERS IN PIXELS WHICH MUST BE RESCALED LATER!!!!!!
    '''
    #get the dims
    N = predictions.shape[0]
    nPixels = np.product(predictions.shape[1:-1])
    nFeatures = predictions.shape[-1]
    
    #reshape array so that it is (N,pixels,features) and thrshold.
    predictions = predictions.reshape((N,nPixels,nFeatures)) > 0.5
    
    #sum of voxels for each 
    areas = np.sum(predictions,axis = 1)
    
    #mean, returning a value for each segmentation channel
    mu = np.mean(areas,axis=0)
    sigma = np.std(areas,axis=0)
    
    return mu,sigma

def predict_stochastic(model,N,X):
    
    '''draw and summarise multiple predictions from a model
    Arguments:
        model {a model, for example a Keras model, with a predict method} -- is assumed to have some stochastic component, i.e. multiple
        N {int} -- the number of sample predictions to be drawn from the stochastic model
        X {numpy array, probably float} -- assumed to be already consistent with inputs to the model. MUST ONLY BE A SINGLE IMAGE AND NOT MULTIPLE STACKED!!!!!
        
    Returns:
        consensus {numpy array, boolean} -- pixelwise segmentation of x
        also various floats, representing different metrics for uncertainty and the outputs.
    '''
    
    #draw N predictions from the model over x
    predictions = np.stack([model.predict(X) for n in range(N)],axis=0)
    
    #binarise
    predictions = predictions
    
    consensus = np.mean(predictions,axis=0)>0.5 
    
    #metrics described in Roy et al...
    uncertainty = voxel_uncertainty(predictions)
    
    mpDsc = mean_pairwise_dsc(predictions)
    gDsc = global_dsc(predictions)
    
    mpIou = mean_pairwise_iou(predictions)
    gIou = global_iou(predictions)
    meanArea,stdArea = mean_std_area(predictions)
    
    return consensus,uncertainty,meanArea,stdArea,mpDsc,gDsc,mpIou,gIou

First, lets go through a range of values for N, the number of samples drawn in making a prediction, before manually selecting an optimal value.

In [ ]:
metricNames = ['mean pairwise Dice coefficient',
               'global Dice coefficient',
               'mean pairwise IOU',
               'global IOU'
              ]

trueNames = ['true IOU','true DSC']

names = ['/'.join((m,t)) for t,m in itertools.product(trueNames,metricNames)]

In [ ]:
from IPython.display import clear_output

In [ ]:
NRange = np.arange(3,21,1)
nAttempts = 25#number of times to run analysis - captures variability in r values and mean IOU/DSC over attempts, rather than each instance...
dscMean = np.zeros((*NRange.shape,nAttempts))
uncertaintyMean = np.zeros((*NRange.shape,nAttempts))
dscStd = np.zeros(NRange.shape)
uncertaintyStd = np.zeros(NRange.shape)
areaStdMean = np.zeros((*NRange.shape,nAttempts))
areaStdStd = np.zeros(NRange.shape)

# U = np.zeros((NRange.shape[0],nAttempts,*Y_test.shape))

r = np.zeros((NRange.shape[0],len(names),nAttempts))

for Nind,N in enumerate(NRange):
    for attempt in range(nAttempts):
        clear_output()
        print('-'.join((str(N),str(attempt))))
        predTest,uncertaintyTest,meanAreaTest,stdAreaTest,mpDscTest,gDscTest,mpIouTest,gIouTest = map(np.array,zip(*[predict_stochastic(model,N,x.reshape(imShape)) for x in X_test]))

        predTest = predTest.reshape(*Y_test.shape)
        uncertaintyTest = uncertaintyTest.reshape(*Y_test.shape)
        #loop over the example axis, calculating metrics for each image separately
        TestDSC = [dsc(Y_test[m,:,:,:], predTest[m,:,:]) for m in range(MTest)]

        #sum of entropy over segmented pixels
        totalUncertainty = [np.sum(u) for u in uncertaintyTest.squeeze()]
        
        dscMean[Nind,attempt] = np.mean(TestDSC)
        uncertaintyMean[Nind,attempt] = np.nanmean(totalUncertainty)

        dscStd[Nind] = np.std(TestDSC)
        uncertaintyStd[Nind] = np.std(totalUncertainty)

        areaStdMean[Nind,attempt] = stdAreaTest.mean()
        areaStdStd[Nind] = stdAreaTest.std()
#         U[Nind,attempt,:,:,:,:] = uncertaintyTest

In [ ]:
dscMean_Deterministic = 0.8005#from previous results with MRUNet/without any ...

#set the graph x location for 
detX = NRange.max() + 2

errorbarArgs = {'capsize':3,
                'marker':'o'
               }

plt.figure(figsize = (15,5))

plt.subplot(1,3,1)
plt.errorbar(NRange,dscMean.mean(axis=-1),dscMean.std(axis=-1),**errorbarArgs)
plt.ylabel('mean Dice')
plt.axhline(dscMean_Deterministic,linestyle = '--',label = 'no dropout',c='r')
plt.legend()
plt.xlabel('Monte Carlo N')
plt.xticks(NRange)

plt.subplot(1,3,2)
plt.errorbar(NRange,areaStdMean.mean(axis=-1),areaStdMean.std(axis=-1),**errorbarArgs)
plt.xlabel('Monte Carlo N')
plt.ylabel('Mean estimated standard deviation of area (cm$^{2}$)')
plt.xticks(NRange)

plt.subplot(1,3,3)
# plt.errorbar(NRange,np.diff(uncertaintyMean.mean(axis=-1)),np.diff(uncertaintyMean.std(axis=-1)),**errorbarArgs)
u = np.diff(uncertaintyMean,axis=0)
plt.errorbar(NRange[1:],np.nanmean(u, axis=-1),np.nanstd(u,axis=-1))
plt.xlabel('Monte Carlo N')
plt.ylabel('Delta uncertainty')
plt.xticks(NRange)


plt.savefig(os.path.join(graphDir,'sample_size_means.png'))
plt.savefig(os.path.join(graphDir,'sample_size_means.svg'))


And so, the correct answer is to use a sample size which gives near-best performance _AND_ spread _AND_ r (if r is unchanged) (less samples is obvs better tho).

In [ ]:
N = 15

In [ ]:
eg = 76

im = X_test[eg].squeeze()

negs = 3
plt.figure(figsize = ((negs + 2)*5,5))
plt.subplot(1,negs+2,1)
plt.imshow(im,cmap = 'gray')
plt.xticks([])
plt.yticks([])


preds = np.zeros((negs,*Y_test.shape[1:]))

for ind in range(negs):
    p = model.predict(im.reshape((1,208,208,1))) > 0.5
    preds[ind,:,:,:] = p
    
    plt.subplot(1,negs+2,ind+2)
    plt.imshow(preds[ind].squeeze(),cmap = 'gray')
    plt.xticks([])
    plt.yticks([])

plt.subplot(1,negs+2,negs+2)
show_image_with_masks(im,[p.squeeze() for p in preds],[{'linewidth':1}]*negs)
plt.savefig(os.path.join(graphDir, 'example_seg_mask' + str(eg) + '.svg'))

Now, we can quantify model performance...

In [ ]:
predTest,uncertaintyTest,meanAreaTest,stdAreaTest,mpDscTest,gDscTest,mpIouTest,gIouTest = map(np.array,zip(*[predict_stochastic(model,N,x.reshape(imShape)) for x in X_test]))
predTest = predTest.reshape(*Y_test.shape)

In [ ]:

#loop over th eexample axis, calculating metrics for each image separately
TestIOU = np.array([iou(Y_test[m,:,:,:], predTest[m,:,:]) for m in range(MTest)])
TestDSC = np.array([dsc(Y_test[m,:,:,:], predTest[m,:,:]) for m in range(MTest)])
TestHD = np.array([symmetric_hausdorff_distance(Y_test[m,:,:,:], predTest[m,:,:],pxSpacing_test[m]) for m in range(MTest)])
TestMCD = np.array([mean_contour_distance(Y_test[m,:,:,:], predTest[m,:,:],pxSpacing_test[m]) for m in range(MTest)])


Lets have a look at network performance..

In [ ]:
#Histograms for each of the metrics...

plt.figure(figsize = (20,5))

plt.subplot(1,4,1)
plt.hist(TestIOU ,  bins = np.arange(0,1.05,0.05), density=True, label = 'Test')
plt.xlabel('Intersection-over-Union')
plt.title(f'mean = {np.mean(TestIOU):.3f}, std = {np.std(TestIOU):.3f}')
plt.ylabel('probability density')

plt.subplot(1,4,2)
plt.hist(TestDSC ,  bins = np.arange(0,1.05,0.05), density=True, label = 'Test')
plt.xlabel('Dice score')
plt.title(f'mean = {np.mean(TestDSC):.3f}, std = {np.std(TestDSC):.3f}')

plt.subplot(1,4,4)
plt.hist(TestHD , bins = np.arange(0,125,5), density=True, label = 'Test')
plt.xlabel('Hausdorff Distance (mm)')
plt.title(f'mean = {np.mean(TestHD):.3f}, std = {np.std(TestHD):.3f}')

plt.subplot(1,4,3)
plt.hist(TestMCD , bins = np.arange(0,25,2), density=True, label = 'Test')
plt.xlabel('Mean Contour Distance (mm)')
plt.title(f'mean = {np.mean(TestMCD):.3f}, std = {np.std(TestMCD):.3f}')
# plt.legend()

plt.savefig(os.path.join(graphDir,'metric_histogram.png'))
plt.savefig(os.path.join(graphDir,'metric_histogram.svg'))

In [ ]:
#bland-altman plot for the the test set...
plt.figure(figsize = (5,5))

trueAreas = Y_test.sum(axis = (1,2)) * PXAREA/100
predAreas = predTest.sum(axis = (1,2)) * PXAREA/100

meanArea = (trueAreas + predAreas) /2
diffArea = trueAreas - predAreas

meanDiff = np.mean(diffArea)
stdDiff = np.std(diffArea)

plt.scatter(meanArea,diffArea)

plt.axhline(meanDiff,c='k',alpha = 0.5)
plt.axhline(meanDiff + 1.96*stdDiff,c='k',alpha = 0.5, linestyle = '--')
plt.axhline(meanDiff - 1.96*stdDiff,c='k',alpha = 0.5, linestyle = '--')

plt.title('auto vs manual (n = ' + str(MTest) + ')')

plt.ylim([-40,40])
plt.xlim([0,80])


plt.savefig(os.path.join(graphDir,'bland_altman.png'))
plt.savefig(os.path.join(graphDir,'bland_altman.svg'))

Now, we calculate a few metrics obviously r (already there), but also MAE for categries and MAE for the model predictions!

In [ ]:
#function for categorising DSC

def categorise(metric,lowBoundary=0.6,highBoundary=0.8):
    '''function for categorising DSC, returns an array of same dimension as input, formed of 0,1 and 2. This means that comparison of two such arrays is very simple'''
    bad = metric < lowBoundary
    good = metric >= highBoundary
    medium = ~np.logical_or(bad,good)
    onehot = np.stack((bad,medium,good), axis = -1)
    categories = np.argmax(onehot, axis = -1)
    return categories

In [ ]:
#two ground-truths to be predicted

#4 potential predictions to be made...
plt.figure(figsize =(20,10))

metrics = [mpDscTest,gDscTest,mpIouTest,gIouTest]
metricNames = ['mean pairwise Dice coefficient',
               'global Dice coefficient',
               'mean pairwise IOU',
               'global IOU'
              ]

def scatter_with_title(x,y,bounds):
#     plt.plot([0,1],[0,1],c='k')
    plt.scatter(x,y)
    r,p = pearsonr(x,y)
    lr = LinearRegression()
    lr.fit(x.reshape(-1,1),y)
    pred = lr.predict(x.reshape(-1,1))
    mae = np.mean(np.abs(pred-y))
    
    trueCat = categorise(y,*bounds)
    predCat = categorise(pred,*bounds)
    
    catAcc = (trueCat==predCat).mean()
    plt.title(f'r = {r:.3f}, MAE = {mae:.3f}\ncategory accuracy = {catAcc:.3f}')
    plt.axis('equal')
    plt.plot([0,1],lr.predict(np.array([0,1]).reshape(-1,1)),c='k',label = 'linear fit')
    plt.xlim([0.4,1])
    plt.ylim([0.4,1])

    
for metricInd in range(4):
    plt.subplot(2,4,metricInd+1)
    scatter_with_title(metrics[metricInd],TestDSC,bounds=(0.6,0.8) )
    if metricInd ==0:
        plt.ylabel('True DSC')

    plt.subplot(2,4,5+metricInd)
    scatter_with_title(metrics[metricInd],TestIOU,bounds=(0.4,0.65) )
    if metricInd ==0:
        plt.ylabel('True IOU')
    plt.xlabel(metricNames[metricInd])

plt.savefig(os.path.join(graphDir,'QC_predictions_all_metrics.png'))
plt.savefig(os.path.join(graphDir,'QC_predictions_all_metrics.svg'))

In [ ]:
bestMetric = TestDSC.reshape(-1,1)#either DSC or IOU, depending on the above figures...
bestOutput = mpDscTest.reshape(-1,1)#the best predictor of this metric, based on the above figure

Ensure that this choice of metric/output is propagated to network_utils in order to facilitate scaleup of predictions!

In [ ]:

convertModelFile = modelBaseName + '_prediction_conversion.pickle'
if os.path.isfile(convertModelFile):
    with open(convertModelFile,'rb') as f:
        convertModel = pickle.load(f)
else:
    convertModel = LinearRegression()
    convertModel.fit(bestOutput,bestMetric)
    with open(convertModelFile,'wb') as fOut:
        pickle.dump(convertModel,fOut)
        
predictedAccuracy = convertModel.predict(bestOutput)

According to Roy et al, there are 3 categories of segmentation, bad (DSC < 0.6), medium (DSC >0.6,<0.8) and good (DSC > 0.8). So, lets look at the categories..

Now, lets have a look at what happens when we add noise as a general thing...

Now, we need to show r-values for both real images and those with artificially-introduced noise (see Roy et al)

In [ ]:
def add_rician_noise(v,s):
    
    '''returns a rician-corrupted version of a single image'''
    
    x = np.random.normal(loc=0,scale=s,size=v.shape) + v
    y = np.random.normal(loc=0,scale=s,size=v.shape)
    
    r = np.sqrt(x**2 + y**2)
    return normalise_image(r)    

In [ ]:
eg = 0

im = X_test[eg]

signal = im.mean()

SNRs = np.array([np.inf,4,3,2.5,2])

ncols = SNRs.shape[0]+1

plt.figure(figsize=(5*ncols,10))

for i,SNR in enumerate(SNRs):
    
    scale = signal/SNR
    
    noise = add_rician_noise(im,scale)
    
    pred,uncertainty,meanArea,stdArea,mpDsc,gDsc,mpIou,gIou = predict_stochastic(model,N,noise.reshape(1,208,208,1))
    pred = pred.squeeze()
    
    #show the image
    plt.subplot(2,ncols,i+1)
    show_image_with_masks(image = noise.squeeze(),
                          masks = [Y_test[eg].squeeze(),pred],
                          maskOptions = [{'linewidth':1,'color':'y'},{'linewidth':1,'color':'r'}]
                         )
    plt.xticks([])
    plt.yticks([])

    if i==0:
        plt.ylabel('image and segmentation')
        uncertainties = uncertainty
        plt.title('no added noise')
    else:
        uncertainties = np.concatenate((uncertainties,uncertainty),axis = 0)
        plt.title('SNR = ' + str(SNR))
    #color limits globally
    cl = [0,uncertainties.max()]

for i,SNR in enumerate(SNRs):
    #pixelwise uncertainty
    plt.subplot(2,ncols,ncols+1 + i)
    plt.imshow(uncertainties[i].squeeze(),clim=cl)
    plt.xticks([])
    plt.yticks([])
    if i==0:
        plt.ylabel('pixelwise uncertainty')
#dummy plot to allow colorbar
plt.subplot(2,ncols,2*ncols )
plt.imshow(np.zeros_like(uncertainties[i].squeeze()))
plt.colorbar()

plt.savefig(os.path.join(graphDir,'noise_addition_example.png'))
plt.savefig(os.path.join(graphDir,'noise_addition_example.svg'))

In [ ]:
SNRs = np.array([np.inf,4,3.5,3,2.5])

ncols = SNRs.shape[0]+1

plt.figure(figsize=(5*ncols,10))

NoiseDSC = np.zeros(shape=(SNRs.shape[0],MTest))

mpDscNoise = np.zeros_like(NoiseDSC)

for i,SNR in enumerate(SNRs):
    
    signals = X_test.mean(axis=(1,2))
    
    scales = signals/SNR
    
    X_Noise = [add_rician_noise(im,scale) for im,scale in zip(X_test,scales)]
    
    predNoise,uncertaintyNoise,meanAreaNoise,stdAreaNoise,mpDscNoise[i,:],gDscNoise,mpIouNoise,gIouNoise = map(np.array,zip(*[predict_stochastic(model,N,x.reshape(imShape)) for x in X_Noise]))
    predNoise = predNoise.reshape(*Y_test.shape)
    #loop over theexample axis, calculating DSC for each image
    NoiseDSC[i,:] = np.array([dsc(Y_test[m,:,:,:], predNoise[m,:,:]) for m in range(MTest)])
    
#     mpDscNoise[i,:] = mpDscNoise

In [ ]:
x = mpDscNoise.flatten()
y = NoiseDSC.flatten()

nans = np.isnan(x)
x = x[~nans].reshape(-1,1)
y = y[~nans].reshape(-1,1)

pred = convertModel.predict(x)

r = pearsonr(y.flatten(),pred.flatten())[0]


mae = np.mean(np.abs(pred-y))

trueCat = categorise(y,*bounds)
predCat = categorise(pred,*bounds)

catAcc = (trueCat==predCat).mean()

bounds = (0.6,0.8)

plt.figure(figsize=(5,5))


plt.title(f'r = {r:.3f}, MAE = {mae:.3f}\ncategory accuracy = {catAcc:.3f}')
plt.axis('equal')
plt.plot([0,1],convertModel.predict(np.array([0,1]).reshape(-1,1)),c='k',label = 'linear fit')
for i,(p,t) in enumerate(zip(mpDscNoise,NoiseDSC)):
    if i == 0:
        label = 'no noise added'
    else:
        label = 'SNR = ' + str(SNRs[i])
    plt.scatter(p,t,label = label)

plt.legend()
plt.xlim([0,1])
plt.ylim([0,1])

In [ ]:

negs = 8

egs = np.random.choice(range(MTest), negs, replace=False)

ncols = 4
nrows = np.ceil(negs/ncols)

plt.figure(figsize = (5*ncols,5*nrows))

imShape = X_test.shape[1:-1]

for i in range(negs):
    
    plt.subplot(nrows,ncols,i+1)
    manual,automated = Y_test[egs[i],:,:].reshape(imShape), predTest[egs[i],:,:].reshape(imShape) > 0.5
    
    pxS = pxSpacing_test[egs[i]]
    
    show_image_with_masks(image = X_test[egs[i],:,:].reshape(imShape),
                          masks = [manual,automated],
                          maskOptions = [{'linewidth':1,'color':'y','label':'manual'},{'linewidth':1,'color':'r','label':'automated'}]
                         )
    
    plt.title('Dice = ' + f'{bestMetric[egs[i]][0]:.03}' + '\n' + 
              'predicted Dice = ' + f'{predictedAccuracy[egs[i]][0]:.03}'
              )
plt.legend()

plt.savefig(os.path.join(graphDir,'test_examples.png'))
plt.savefig(os.path.join(graphDir,'test_examples.svg'))